In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "simdata"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [7]:
import torch
import numpy as np
import pickle 
import pandas as pd
import matplotlib.pyplot as plt
import re

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

def find_number(text, c):
    return re.findall(r'%s(\d+)' % c, text)

### Add Experiment

In [70]:
experiment_name = 'SIM, Direct, se2d_spatialxfeaturelinear, 0-0-3'
experiment_comment = 'Directly trained on simulated data with se2d_spatialxfeaturelinear and static_loaders of static0-0-3-preproc0.zip. Varying number of neurons and images.'
fabrikant_name = 'kklurz'

model_fn = "nnsysident.models.models.se2d_spatialxfeaturelinear"
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'


model = pd.DataFrame((Model & 'model_fn="{}"'.format(model_fn)).fetch())
for arg in ['neuron_n', 'image_n']:
    model[arg] = [int(find_number(row.model_comment, arg + '=')[0]) for loc, row in model.iterrows()]
for arg in ['gamma_input', 'gamma_readout']:
    model[arg] = [row.model_config[arg] for loc, row in model.iterrows()]
    
dataset = pd.DataFrame((Dataset & 'dataset_fn="{}"'.format(dataset_fn)).fetch())
for arg in ['neuron_n', 'image_n']:
    dataset[arg] = [row.dataset_config[arg] for loc, row in dataset.iterrows()]
    
combinations = dataset.join(model.set_index(['neuron_n', 'image_n']), on=['neuron_n', 'image_n']).sort_values(['neuron_n', 'image_n'])
experiment = [{'dataset_hash': row['dataset_hash'], 
               'dataset_fn': dataset_fn,
               'model_hash': row['model_hash'],
               'model_fn': model_fn,
               'trainer_hash': 'd41d8cd98f00b204e9800998ecf8427e',
               'trainer_fn': 'nnsysident.training.trainers.standard_trainer',
                 'experiment_name': experiment_name} for loc, row in combinations.iterrows()]

Experiments.insert1(dict(experiment_name=experiment_name, fabrikant_name=fabrikant_name, experiment_comment=experiment_comment))
Experiments.Restrictions.insert(experiment)

In [64]:
Model()& 'model_fn = "nnsysident.models.models.se2d_spatialxfeaturelinear"'

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nnsysident.models.models.se2d_spatialxfeaturelinear,13938643b718b366d0a6a8eb5a73fa16,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=100, image_n=200",2020-05-05 08:48:29
nnsysident.models.models.se2d_spatialxfeaturelinear,284dfc875c379a6b19fde5ef1da5fa5a,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=100, image_n=50",2020-05-05 08:27:54
nnsysident.models.models.se2d_spatialxfeaturelinear,42044b578927adf88c7d2ccd7e15deaa,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=1000, image_n=1000",2020-05-05 08:54:23
nnsysident.models.models.se2d_spatialxfeaturelinear,46dd035e3562088ed02adbf6a3e7810f,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=1000, image_n=50",2020-05-05 11:53:59
nnsysident.models.models.se2d_spatialxfeaturelinear,7bb9706fe458e824951fc3f73b6ab933,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=100, image_n=100",2020-05-05 08:47:42
nnsysident.models.models.se2d_spatialxfeaturelinear,8bbc2bbac0481eec478ea11cbb6371d3,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=1000, image_n=200",2020-05-05 08:36:58
nnsysident.models.models.se2d_spatialxfeaturelinear,954e2724c7c7e2574241db05bd285dbe,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=1000, image_n=100",2020-05-05 08:36:17
nnsysident.models.models.se2d_spatialxfeaturelinear,9c8ae666f76d6c7407489fa242ca5e6f,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=100, image_n=4000",2020-05-05 12:57:44
nnsysident.models.models.se2d_spatialxfeaturelinear,bd14bc7edfafa51ed5354b718fd25ae8,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=100, image_n=500",2020-05-05 09:00:21
nnsysident.models.models.se2d_spatialxfeaturelinear,c3280fc25057d0d251e6f38eccbe5846,=BLOB=,kklurz,"se2d_spatialxfeaturelinear, neuron_n=1000, image_n=4000",2020-05-05 09:57:58


In [65]:
Model & 'model_fn = "nnsysident.models.models.se2d_pointpooled"'

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nnsysident.models.models.se2d_pointpooled,01a31f40c16926dd875aa4f6c934e867,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=100, image_n=500",2020-05-05 12:09:36
nnsysident.models.models.se2d_pointpooled,0d5544ad26c8bfbaf0a68365b130eff2,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=500",2020-05-04 19:20:38
nnsysident.models.models.se2d_pointpooled,205473520e467a848d065aed92cb228d,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=100, image_n=4000",2020-05-04 21:06:19
nnsysident.models.models.se2d_pointpooled,295dd15f07b39de3d2120cee80fce3ea,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=100, image_n=100",2020-05-04 18:13:33
nnsysident.models.models.se2d_pointpooled,2d627b0220eff65b6ebd282ea0f2890e,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=100, image_n=1000",2020-05-04 18:58:18
nnsysident.models.models.se2d_pointpooled,54c36b4408f121362d3622c1b5310acc,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=4000",2020-05-04 22:36:35
nnsysident.models.models.se2d_pointpooled,58ae88c0a146d53e24f01695ab579548,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=100",2020-05-04 18:41:38
nnsysident.models.models.se2d_pointpooled,77109b22411129b7d7c5f7ab1f47127a,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=1000",2020-05-04 20:05:51
nnsysident.models.models.se2d_pointpooled,c3ad079c7f5d7b575683eb157ef10b6b,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=200",2020-05-04 18:29:16
nnsysident.models.models.se2d_pointpooled,d16da535e4d2d1c74ccce10e23dab795,=BLOB=,kklurz,"se2d_pointpooled, neuron_n=1000, image_n=50",2020-05-04 18:35:24
